<a href="https://colab.research.google.com/github/Hongjieun123/deeplearning-project/blob/master/CNN_example_3_1(first_conv_layer_4x4%2C_second_conv_layer_4x4_third_conv_4x4_%2Cfilter_num_%3D_32%2C32%2C32).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab에서 tensorflow 1.x 실행
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [17]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

print('\n', mnist.train.num_examples, mnist.test.num_examples, mnist.validation.num_examples)

print('\ntrain image shape = ', np.shape(mnist.train.images))
print('train label shape = ', np.shape(mnist.train.labels))
print('test image shape = ', np.shape(mnist.test.images))
print('test label shape = ', np.shape(mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

 55000 10000 5000

train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)


In [0]:
learning_rate = 0.001
epochs = 30
batch_size = 100

In [0]:
X = tf.placeholder(tf.float32, [None, 784])

A1 = X_img = tf.reshape(X,[-1, 28, 28, 1])

T = tf.placeholder(tf.float32, [None, 10])  # image 28 X 28 X 1 (black/white)

In [0]:
# 1번째 컨볼루션 층
# 4X4 크기를 가지는 32개의 필터를 적용

F2 = tf.Variable(tf.random_normal([4, 4, 1, 32], stddev=0.01))  
b2 = tf.Variable(tf.constant(0.1, shape=[32]))   

# 1번째 컨볼루션 연산을 통해 28 X 28 X1  => 28 X 28 X 32
C2 = tf.nn.conv2d(A1, F2, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z2 = tf.nn.relu(C2+b2)

# 1번째 max pooling을 통해 28 X 28 X 32  => 14 X 14 X 32
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 2번째 컨볼루션 층
# 4X4 크기를 가지는 32개의 필터를 적용

F3 = tf.Variable(tf.random_normal([4, 4, 32, 32], stddev=0.01))  
b3 = tf.Variable(tf.constant(0.1, shape=[32]))   

# 2번째 컨볼루션 연산을 통해 14 X 14 X32  => 14 X 14 X 32
C3 = tf.nn.conv2d(A2, F3, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z3= tf.nn.relu(C3+b3)

# 2번째 max pooling을 통해 14 X 14 X 32  => 7 X 7 X 32
A3= P3 = tf.nn.max_pool(Z3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 3번째 컨볼루션 층
# 4X4 크기를 가지는 32개의 필터를 적용

F4 = tf.Variable(tf.random_normal([4, 4, 32, 32], stddev=0.01))  
b4 = tf.Variable(tf.constant(0.1, shape=[32]))   

# 3번째 컨볼루션 연산을 통해 7 X 7 X32  => 7 X 7 X 32
C4 = tf.nn.conv2d(A3, F4, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z4= tf.nn.relu(C4+b4)

# 3번째 max pooling을 통해 7 X 7 X 32  => 4 X 4 X 32
A4= P4 = tf.nn.max_pool(Z4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 4X4 크기를 가진 32개의 activation map을 flatten 시킴
A4_flat = P4_flat = tf.reshape(A4, [-1, 32*4*4])

In [0]:
# 출력층
W5 = tf.Variable(tf.random_normal([32*4*4, 10], stddev=0.01))
b5 = tf.Variable(tf.random_normal([10]))

# 출력층 선형회귀  값 Z5, 즉 softmax 에 들어가는 입력 값
Z5 = logits = tf.matmul(A4_flat, W5) + b5    # 선형회귀 값 Z5

y = A5 = tf.nn.softmax(Z5)

In [0]:
loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z5, labels=T) )

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [0]:
# batch_size X 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal( tf.argmax(A5, 1), tf.argmax(T, 1) )

# batch_size X 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

# index list 출력
accuracy_index = tf.cast(predicted_val, dtype=tf.float32)

# 예측값 처리
predicted_list = tf.argmax(A5, 1)

In [34]:
index_label_prediction_list = []
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화
    
    start_time = datetime.now()
    
    for i in range(epochs):    # 30 번 반복수행
        
        total_batch = int(mnist.train.num_examples / batch_size)  # 55,000 / 100

        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
      
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now() 
    
    print("\nelapsed time = ", end_time - start_time) 
    
    # Accuracy 확인
    test_x_data = mnist.test.images    # 10000 X 784
    test_t_data = mnist.test.labels    # 10000 X 10
    
    accuracy_val, predicted_list_val, index_label = sess.run([accuracy, predicted_list, accuracy_index], feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy = ", accuracy_val)
    print("type(accuracy_val) = ", type(accuracy_val), ', type(predicted_list_val) = ', type(predicted_list_val), ', type(index_label) = ', type(index_label))
    print("index_label.shape = ", index_label.shape)
    
    index_label_list = list(index_label)
    print("length of index_label_list = ", len(index_label_list))
    print("false label count = ", index_label_list.count([0]))
        
    
    temp_list = [] 
    
    for index in range(len(index_label)):
        
        if index_label[index] == 0:
            
            temp_list.append(index)
            temp_list.append(np.argmax(test_t_data[index]))  # one-hot encoding 이므로 argmax 로 정답 추출
            temp_list.append(predicted_list_val[index])
            
            index_label_prediction_list.append(temp_list)
            
            temp_list = []
            
    print("\nlength of index_label_false_list", len(index_label_prediction_list))

epochs =  0 , step =  0 , loss_val =  2.655664
epochs =  0 , step =  100 , loss_val =  2.3022137
epochs =  0 , step =  200 , loss_val =  0.6538093
epochs =  0 , step =  300 , loss_val =  0.2896741
epochs =  0 , step =  400 , loss_val =  0.21059498
epochs =  0 , step =  500 , loss_val =  0.10466814
epochs =  1 , step =  0 , loss_val =  0.14368269
epochs =  1 , step =  100 , loss_val =  0.08838283
epochs =  1 , step =  200 , loss_val =  0.06059802
epochs =  1 , step =  300 , loss_val =  0.08190482
epochs =  1 , step =  400 , loss_val =  0.063102715
epochs =  1 , step =  500 , loss_val =  0.11592269
epochs =  2 , step =  0 , loss_val =  0.038839754
epochs =  2 , step =  100 , loss_val =  0.075082295
epochs =  2 , step =  200 , loss_val =  0.044092193
epochs =  2 , step =  300 , loss_val =  0.062302977
epochs =  2 , step =  400 , loss_val =  0.058928657
epochs =  2 , step =  500 , loss_val =  0.07840989
epochs =  3 , step =  0 , loss_val =  0.04117313
epochs =  3 , step =  100 , loss_val =